# FedHealth-J — One-Click Colab Demo
This notebook is a self-contained demo of **FedHealth-J**:
- clones the repo (if not present),
- installs minimal dependencies,
- generates the synthetic **J-Health100** dataset,
- applies Japanese cultural/seasonal adjustments,
- simulates 5 federated clients training an LSTM autoencoder locally,
- performs simple federated averaging,
- computes anomaly (reconstruction) scores and prints example JSON alerts.

**Run each cell in order.** Designed for reviewers/professors — no local setup required.

In [ ]:
!pip install --quiet numpy pandas tensorflow scikit-learn
import os, sys, subprocess
REPO_URL = "https://github.com/avartan007/fedhealth-j.git"
REPO_DIR = "/content/fedhealth-j"
if not os.path.isdir(REPO_DIR):
    print("Cloning repository...")
    subprocess.run(["git", "clone", REPO_URL, REPO_DIR])
sys.path.insert(0, REPO_DIR)
os.listdir(REPO_DIR)

In [ ]:
from importlib import reload
import j_health_100_generator as gen
import cultural_profile as cp
reload(gen); reload(cp)
print("Imported local modules")

In [ ]:
df = gen.generate_j_health_100()
print("Generated dataset size:", len(df))
df.head(6)

In [ ]:
import numpy as np
adjusted = []
for _, r in df.iterrows():
    profile = cp.CulturalProfile(r['region'], r['season'])
    hr = profile.adjust_heart_rate(float(r['avg_heart_rate']))
    steps = profile.adjust_steps(int(r['daily_steps']))
    sleep = float(r['sleep_hours'])
    adjusted.append([hr, steps, sleep])
X = np.array(adjusted)
TIMESTEPS = 10
X_seq = np.repeat(X[:, np.newaxis, :], TIMESTEPS, axis=1)
print("Prepared sequence data shape:", X_seq.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam
def create_lstm_autoencoder(timesteps=TIMESTEPS, features=3, latent_dim=16):
    model = Sequential([
        LSTM(32, activation='relu', input_shape=(timesteps, features), return_sequences=True),
        LSTM(latent_dim, activation='relu', return_sequences=False),
        RepeatVector(timesteps),
        LSTM(latent_dim, activation='relu', return_sequences=True),
        LSTM(32, activation='relu', return_sequences=True),
        TimeDistributed(Dense(features))
    ])
    model.compile(optimizer=Adam(1e-3), loss='mse')
    return model
m = create_lstm_autoencoder()
m.summary()

In [ ]:
NUM_CLIENTS = 5
client_splits = np.array_split(X_seq, NUM_CLIENTS)
client_models = []
client_weights = []
client_sizes = []
for idx, data in enumerate(client_splits):
    model = create_lstm_autoencoder()
    model.fit(data, data, epochs=1, batch_size=8, verbose=0)
    client_models.append(model)
    client_weights.append(model.get_weights())
    client_sizes.append(len(data))
    print(f"Client {idx+1} trained on {len(data)} samples.")
def weighted_average(weights_list, sizes):
    avg = []
    total = float(sum(sizes))
    for layer_weights in zip(*weights_list):
        layer_sum = np.zeros_like(layer_weights[0])
        for w, s in zip(layer_weights, sizes):
            layer_sum += w * (s/total)
        avg.append(layer_sum)
    return avg
global_weights = weighted_average(client_weights, client_sizes)
global_model = create_lstm_autoencoder()
global_model.set_weights(global_weights)
print("Federated averaging complete. Global model ready.")

In [ ]:
import json
from datetime import datetime
recon = global_model.predict(X_seq, verbose=0)
mse_per_sample = np.mean(np.square(recon - X_seq), axis=(1,2))
threshold = mse_per_sample.mean() + 1.5 * mse_per_sample.std()
anomaly_indices = np.where(mse_per_sample > threshold)[0]
alerts = []
for idx in anomaly_indices[:5]:
    uid = df.loc[idx, 'user_id']
    reason = []
    if df.loc[idx, 'avg_heart_rate'] > df['avg_heart_rate'].mean() + 5:
        reason.append("Elevated heart rate vs baseline")
    if df.loc[idx, 'daily_steps'] < df['daily_steps'].mean() - 1000:
        reason.append("Low activity (steps)")
    if df.loc[idx, 'sleep_hours'] < 5.5:
        reason.append("Short sleep duration")
    alert = {
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "user_id": uid,
        "anomaly_score": float(mse_per_sample[idx]),
        "reason": "; ".join(reason) if reason else "Unusual multivariate pattern",
        "recommendation": "Please check vitals and recent environment (heat/cold/fall risk)"
    }
    alerts.append(alert)
print("Anomaly threshold (demo):", float(threshold))
print("Number of anomalies detected (demo):", int(len(anomaly_indices)))
print(json.dumps(alerts, indent=2))

In [ ]:
OUT_PATH = "/content/fedhealth-j/sample_anomalies.json"
with open(OUT_PATH, "w") as f:
    json.dump(alerts, f, indent=2)
print("Saved sample anomaly file to:", OUT_PATH)

## Done — what you just ran
- Synthetic dataset generated: `j_health_100.csv`
- Data adjusted using cultural profiles
- Local federated training simulated (5 clients)
- Global model produced via simple federated averaging
- Anomaly scores computed and sample JSON alerts saved to `sample_anomalies.json`

Professors can click **Run All** directly in Colab. This notebook is self-contained.